In [2]:
import _init_paths
import os, sys, cv2, pickle
import os.path as osp
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy.random as npr
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances

from Config import Config
from datasets.Pascal import Pascal
from datasets.ROIGenerator import ROIGenerator
from nets.VGG16_SSDH1 import VGG16_SSDH1
from nets.VGG16_FT import VGG16_FT
from retriever.dist_tools import *

In [3]:
# setup & load configs
_C = Config(config_pn="../config/config.ini")
cfg = _C.cfg

In [4]:
ds_name = "voc_2007"
#model_pn = "../../0-bak/20180804_voc_2007_SSDH.WL2/outputs/voc_2007/train_VGG16/VGG16_SSDH_iter_9396_acc_0.907.ckpt" # try 4
#model_pn = "../outputs_G1/voc_2007/train_VGG16/VGG16_SSDH_iter_6264_acc_0.955.ckpt"
model_pn = "../outputs/voc_2007/train_VGG16/VGG16_SSDH1_iter_5024_acc_0.953.ckpt"

In [5]:
dataset_DIR = osp.join(cfg.MAIN_DIR_ROOT, "data", ds_name)
dataset = Pascal(name=ds_name, path_dir=dataset_DIR, cfg=cfg)
dataset.load_sets()
print ('[INFO] dataset.name: {}'.format(dataset.name))
print ('[INFO] dataset.num_cls: {}'.format(dataset.num_cls))
print ('[INFO] dataset.train: {}'.format(dataset.sets["train"]["num_items"]))
print ('[INFO] dataset.trainval: {}'.format(dataset.sets["trainval"]["num_items"]))
print ('[INFO] dataset.test: {}'.format(dataset.sets["test"]["num_items"]))
print ('[INFO] dataset.val: {}'.format(dataset.sets["val"]["num_items"]))

[INFO] dataset.name: voc_2007
[INFO] dataset.num_cls: 21
[INFO] dataset.train: 2501
[INFO] dataset.trainval: 5011
[INFO] dataset.test: 4952
[INFO] dataset.val: 2510


In [6]:
train_images, pos_weights = dataset.load_gt_rois(gt_set="trainval")
train_images = [image for image in train_images if not image.rois["gt"]['flipped']]
val_images, _ = dataset.load_gt_rois(gt_set="test")
print ('[INFO] train_images.num: {}'.format(len(train_images)))
print ('[INFO] val_images.num: {}'.format(len(val_images)))

[INFO] loading gt rois for voc_2007...
[INFO] images with gt loaded from /home/rfm/eclipse/python/region-dh/cache/voc_2007_gt_trainval_images.pkl
[INFO] loading gt rois for voc_2007...
[INFO] images with gt loaded from /home/rfm/eclipse/python/region-dh/cache/voc_2007_gt_test_images.pkl
[INFO] train_images.num: 5011
[INFO] val_images.num: 4952


In [7]:
tfconfig = tf.ConfigProto(allow_soft_placement=True)
tfconfig.gpu_options.allow_growth=True
# init session
sess = tf.Session(config=tfconfig)

In [8]:
cfg.TEST_BATCH_CFC_NUM_IMG = 1
num_bits = 48
net = VGG16_SSDH1(cfg, num_bits, multilabel=True)
#net = VGG16_FT(cfg, multilabel=True)
net.create_architecture('TEST', dataset.num_cls, tag='default')

cls_score.shape:  (1, 20, 2)
cls_prob.shape:  (1, 20, 2)
cls_pred.shape:  (1, 20, 1)


{'cls_score': <tf.Tensor 'vgg_16_2/cls_score:0' shape=(1, 20, 2) dtype=float32>,
 'cls_prob': <tf.Tensor 'vgg_16_2/cls_prob:0' shape=(1, 20, 2) dtype=float32>,
 'cls_pred': <tf.Tensor 'vgg_16_2/cls_pred:0' shape=(1, 20, 1) dtype=int64>,
 'fc7': <tf.Tensor 'Reshape:0' shape=(1, 4096) dtype=float32>,
 'fc_emb': <tf.Tensor 'Reshape_1:0' shape=(1, 48) dtype=float32>}

In [9]:
print(('Loading model check point from {:s}').format(model_pn))
saver = tf.train.Saver()
saver.restore(sess, model_pn)
print('Loaded.')

Loading model check point from ../outputs/voc_2007/train_VGG16/VGG16_SSDH1_iter_5024_acc_0.953.ckpt
INFO:tensorflow:Restoring parameters from ../outputs/voc_2007/train_VGG16/VGG16_SSDH1_iter_5024_acc_0.953.ckpt
Loaded.


In [10]:
def im_classify(sess, net, image, data_gen):
    random_scale_inds = npr.randint(0, high=len(cfg.TEST_DEFAULT_SCALES),size=1)
    
    im_blob, im_scales = data_gen.built_image_blob([image], random_scale_inds)    
    im_info = np.array([im_blob.shape[1], im_blob.shape[2], im_scales[0]], dtype=np.float32)    
    blobs = {"data": im_blob, "im_info": im_info}
    
    cls_score, cls_prob, cls_pred, fc_emb, fc7 = net.test_image(sess, blobs)
    
    return cls_score, cls_prob, cls_pred, fc_emb, fc7

def lp_to_names(lp, ds):
    id_to_cls = {v:k for (k,v) in ds.cls_to_id.items()}
    names = []
    for i, b in enumerate(lp):
        if b == 1:
            names.append(id_to_cls[i+1])
    
    return names

In [11]:
val_gen   = ROIGenerator(val_images, dataset, None, cfg)
train_gen = ROIGenerator(train_images, dataset, None, cfg)

#### Data Analysis

In [12]:
filenames = []
pathnames = []
all_lp = []
all_labels = []
cls_probs = []
cls_preds = []
fc_embs = []
deep_feats = []



for i, image in enumerate(val_images[:10]):
    if i%500 == 0:
        print("processed {}/{}".format(i, len(val_images)))
    
    #raise NotImplementedError
    try:
        cls_score, cls_prob, cls_pred, fc_emb, fc7 = im_classify(sess, net, image, val_gen)
        #pass
    except:
        print("[ALERT] exception on ", image.filename)
        continue
    filenames.append(image.filename)
    pathnames.append(image.pathname)
    lp = image.rois["gt"]["labels"]
    all_lp.append(lp)
    labels = lp_to_names(lp, dataset)
    all_labels.append(labels)    
    cls_probs.append(cls_prob[0])
    cls_preds.append(cls_pred[0])
    fc_embs.append(fc_emb[0])
    deep_feats.append(fc7[0])

processed 0/4952


In [13]:
data = {"filename": filenames, "pathname": pathnames, 
       "labels": all_labels, "lp": all_lp, 
        "cls_pred": cls_preds, "cls_prob": cls_probs,
        "fc_emb": fc_embs, "fc7": deep_feats}
df_val_images = pd.DataFrame(data)

In [14]:
len(df_val_images)

10

In [15]:
df_val_images.head(10)

,filename,pathname,labels,lp,cls_pred,cls_prob,fc_emb,fc7
0,000001.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,"[dog, person]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[0.9992331, 0.0007668677], [0.99066156, 0.009...","[0.062033392, 0.9991524, 0.98536545, 0.8855191...","[0.0, 1.7489784, 0.027950786, 0.0, 0.0, 0.0, 0..."
1,000002.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,[train],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[0.9693308, 0.030669222], [0.98363566, 0.0163...","[0.94086075, 0.021649025, 0.006490744, 0.01212...","[1.6574028, 1.760202, 0.024582172, 0.021275401..."
2,000003.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,"[chair, sofa]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [1], [0], [0], ...","[[0.99496645, 0.005033529], [0.9270546, 0.0729...","[0.7677578, 0.021582494, 0.32228443, 0.0963709...","[0.8821423, 0.70566946, 1.4334462, 0.0, 0.6353..."
3,000004.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,[car],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [1], [0], [0], ...","[[0.99502724, 0.0049727764], [0.955569, 0.0444...","[0.9916453, 0.0002975352, 0.0066284793, 0.4692...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7616861,..."
4,000006.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,"[chair, diningtable, pottedplant]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [1], ...","[[0.9979633, 0.0020366781], [0.9951567, 0.0048...","[1.0521643e-05, 0.99989235, 0.99015254, 0.0003...","[0.6179421, 3.4826717, 0.0, 0.0, 0.0, 0.0, 0.0..."
5,000008.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,[chair],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [1], ...","[[0.9988111, 0.0011889], [0.9932233, 0.0067766...","[0.00029561477, 0.9995098, 0.98685116, 0.27422...","[0.0, 0.69737256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
6,000010.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,"[horse, person]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[0.98786724, 0.012132747], [0.9713618, 0.0286...","[0.9158835, 0.014124763, 0.73582923, 0.9999304...","[0.0, 0.0, 0.16516474, 0.0, 0.0, 0.0, 0.625433..."
7,000011.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,[cat],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [1], [0], ...","[[0.992184, 0.007816067], [0.9970687, 0.002931...","[0.88111454, 0.9996916, 0.99611914, 0.15701412...","[0.0, 0.17468472, 2.57124, 0.0, 1.8176091, 0.0..."
8,000013.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,[cow],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[0.9857663, 0.014233694], [0.99679357, 0.0032...","[0.99989176, 0.9978721, 1.9263342e-05, 0.00076...","[0.0, 0.0, 0.28673437, 2.2741349, 0.16053562, ..."
9,000014.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,"[bus, car, person]","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, ...","[[0], [0], [0], [0], [0], [0], [1], [0], [0], ...","[[0.9949732, 0.0050268085], [0.98050094, 0.019...","[0.99851304, 0.0001071809, 0.00034636518, 0.08...","[0.0, 0.0, 0.0, 1.1848519, 0.018389458, 0.0, 0..."


In [40]:
with open("../data/voc_2007/20180810_df_val_images_SSDH.wl2.try-memb1.pkl",'wb') as fp:
    pickle.dump(df_val_images, fp)

In [16]:
df_val_images[df_val_images["filename"] == "000006.jpg"]

,filename,pathname,labels,lp,cls_pred,cls_prob,fc_emb,fc7
4,000006.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,"[chair, diningtable, pottedplant]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [1], ...","[[0.9979633, 0.0020366781], [0.9951567, 0.0048...","[1.0521643e-05, 0.99989235, 0.99015254, 0.0003...","[0.6179421, 3.4826717, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [17]:
fc_emb1 = list(df_val_images[df_val_images["filename"] == "000006.jpg"]["fc_emb"])[0]
fc_hash1 = np.where(fc_emb1 >= 0.5, 1, 0)
#fc_hash1 = np.reshape(fc_hash1, [-1])
#print(fc_emb1[8])
print(fc_hash1)

[0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1
 0 0 0 0 0 1 1 1 1 1 0]


In [17]:
fc_emb1 = list(df_val_images[df_val_images["filename"] == "000006.jpg"]["fc_emb"])[0]
fc_hash1 = np.where(fc_emb1 >= 0.5, 1, 0)
fc_hash1 = np.reshape(fc_hash1, [-1])
#print(fc_emb1[8])
print(fc_hash1)

[0 1 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1
 0 0 0 0 0 1 1 1 1 1 0]


In [20]:
df_val_images[df_val_images["filename"] == "000008.jpg"]

,filename,pathname,labels,lp,cls_pred,cls_prob,fc_emb,fc7
5,000008.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,[chair],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [1], ...","[[0.9988111, 0.0011889], [0.9932233, 0.0067766...","[0.00029561477, 0.9995098, 0.98685116, 0.27422...","[0.0, 0.69737256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [21]:
fc_emb2 = list(df_val_images[df_val_images["filename"] == "000008.jpg"]["fc_emb"])[0]
fc_hash2 = np.where(fc_emb2 >= 0.5, 1, 0)
fc_hash2 = np.reshape(fc_hash2, [-1])
#print(fc_emb2[8])
print(fc_hash2)

[0 1 1 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 1 1 0 1 0 0 1
 0 0 0 0 0 1 1 1 1 1 0]


In [22]:
df_val_images[df_val_images["filename"] == "000002.jpg"]

,filename,pathname,labels,lp,cls_pred,cls_prob,fc_emb,fc7
1,000002.jpg,/home/rfm/eclipse/python/region-dh/data/voc_20...,[train],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[0.9693308, 0.030669222], [0.98363566, 0.0163...","[0.94086075, 0.021649025, 0.006490744, 0.01212...","[1.6574028, 1.760202, 0.024582172, 0.021275401..."


In [23]:
fc_emb3 = list(df_val_images[df_val_images["filename"] == "000002.jpg"]["fc_emb"])[0]
fc_hash3 = np.where(fc_emb3 >= 0.5, 1, 0)
fc_hash3 = np.reshape(fc_hash3, [-1])
#print(fc_emb2[8])
print(fc_hash3)

[1 0 0 0 1 0 0 1 0 1 0 1 1 1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 0 1 0
 1 0 0 1 1 0 1 1 0 1 0]


In [24]:
#pairwise_distances(np.array([fc_hash1[8]]), np.array([fc_hash2[8]]), 'hamming')[0]
pairwise_distances(np.array([fc_hash1]), np.array([fc_hash2]), 'hamming')[0]

array([0.10416667])

In [25]:
#pairwise_distances(np.array([fc_hash1[8]]), np.array([fc_hash3[8]]), 'hamming')[0]
pairwise_distances(np.array([fc_hash1]), np.array([fc_hash3]), 'hamming')[0]

array([0.47916667])

In [26]:
#pairwise_distances(np.array([fc_hash2[8]]), np.array([fc_hash3[8]]), 'hamming')[0]
pairwise_distances(np.array([fc_hash2]), np.array([fc_hash3]), 'hamming')[0]

array([0.54166667])